# 날씨 크롤링

In [1]:
import json
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from ast import literal_eval
import pandas as pd
import json
from datetime import datetime
from pytz import timezone
import time
import warnings
# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [2]:
today = datetime.now(timezone('Asia/Seoul'))

In [3]:
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/위도%2C경도?unitGroup=metric&key=APIKEY&include=fcst%2Chours'
url = url.replace('위도','37.61950861037027')
url = url.replace('경도','127.05806240064898')
url = url.replace('APIKEY','발급받은 APIKEY 넣기')

In [4]:
response = requests.get(url)
data = response.json()
days = data['days']

In [5]:
df = pd.DataFrame(columns = ['date'])
for i in range(0,len(days)):
    for j in range(0,len(days[i]['hours'])):
        df = df.append(days[i]['hours'][j],ignore_index=True)
    df['date'].fillna(days[i]['datetime'],inplace = True)
df

,date,cloudcover,conditions,datetime,datetimeEpoch,dew,feelslike,humidity,icon,precip,...,solarenergy,solarradiation,source,stations,temp,uvindex,visibility,winddir,windgust,windspeed
0,2021-07-22,100.0,Overcast,00:00:00,1.626880e+09,21.3,30.4,66.1,cloudy,0.0,...,0.0,0.0,fcst,None,28.2,0.0,24.1,157.8,2.9,2.5
1,2021-07-22,100.0,Overcast,01:00:00,1.626883e+09,21.1,29.7,67.1,cloudy,0.0,...,0.0,0.0,fcst,None,27.7,0.0,24.1,35.5,2.9,2.9
2,2021-07-22,100.0,Overcast,02:00:00,1.626887e+09,20.9,28.9,68.0,cloudy,0.0,...,0.0,0.0,fcst,None,27.2,0.0,24.1,104.9,2.9,2.9
3,2021-07-22,100.0,Overcast,03:00:00,1.626890e+09,20.6,26.7,69.3,cloudy,0.0,...,0.0,0.0,fcst,None,26.7,0.0,24.1,76.4,3.2,3.2
4,2021-07-22,100.0,Overcast,04:00:00,1.626894e+09,20.5,26.2,70.5,cloudy,0.0,...,0.0,0.0,fcst,None,26.2,0.0,24.1,51.8,3.6,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2021-08-05,3.3,Clear,19:00:00,1.628158e+09,21.9,30.6,70.0,clear-day,0.0,...,1.2,330.3,fcst,None,27.9,3.0,24.1,37.1,14.0,8.9
356,2021-08-05,2.5,Clear,20:00:00,1.628161e+09,22.2,29.3,74.5,clear-night,0.0,...,0.9,259.0,fcst,None,27.1,3.0,24.1,27.9,10.4,6.8
357,2021-08-05,22.5,Partially cloudy,21:00:00,1.628165e+09,22.1,28.8,76.0,partly-cloudy-night,0.0,...,0.6,172.7,fcst,None,26.7,2.0,24.1,25.8,10.6,6.6
358,2021-08-05,42.4,Partially cloudy,22:00:00,1.628168e+09,22.0,26.3,77.6,partly-cloudy-night,0.0,...,0.3,86.3,fcst,None,26.3,1.0,24.1,23.7,10.7,6.4


In [6]:
result = df[['date','datetime','temp','windspeed','precip','precipprob','humidity']]
result.rename(columns={'datetime':'hour'},inplace = True)

In [7]:
result['hour']=result['hour'].str[:2]
result = result.astype({'hour':'int'})
result

,date,hour,temp,windspeed,precip,precipprob,humidity
0,2021-07-22,0,28.2,2.5,0.0,0.0,66.1
1,2021-07-22,1,27.7,2.9,0.0,0.0,67.1
2,2021-07-22,2,27.2,2.9,0.0,0.0,68.0
3,2021-07-22,3,26.7,3.2,0.0,0.0,69.3
4,2021-07-22,4,26.2,3.6,0.0,0.0,70.5
...,...,...,...,...,...,...,...
355,2021-08-05,19,27.9,8.9,0.0,23.7,70.0
356,2021-08-05,20,27.1,6.8,0.0,23.7,74.5
357,2021-08-05,21,26.7,6.6,0.0,4.8,76.0
358,2021-08-05,22,26.3,6.4,0.0,4.8,77.6


In [8]:
result.to_csv("./weather_{}.csv".format(datetime.strftime(today,"%y%m%d")),index=False,encoding='cp949')